# This is a scratch space for building functions
After functions are made, move them to the corresponding folder. 

In [ ]:
from pathlib import Path
from flow_analysis_comps.PIV.PIV_process import AMF_PIV
from flow_analysis_comps.PIV.definitions import PIV_params
from flow_analysis_comps.PIV.PIV_visualize import PIV_visualize, visualizerParams
import matplotlib.pyplot as plt

%matplotlib widget
%load_ext autoreload
%autoreload 2

In [ ]:
input_adr = Path(r"G:\AMOLF_Data\AMOLF-SHIMIZU Dropbox\Simon van Staalduine\vortex_stable\OpenPIV_results_8_PIV_output")
params = visualizerParams(use_mask=False, use_flags=True)
PIV_visualizer = PIV_visualize(input_adr, parameters=params)

In [ ]:
PIV_visualizer.set_image_index(345)
PIV_visualizer.plot_vortex_potential(30, data_shown="vortex", index_stop = -1, speed_threshold=1e-1)

In [ ]:
PIV_visualizer.show_current_frame()

In [ ]:
data_adr = Path(r"G:\AMOLF_Data\AMOLF-SHIMIZU Dropbox\Simon van Staalduine")
plate_id = r"20250122_Plate017"
video_id = r"vortex_stable"
frame_id1 = r"Img_0000.tif"
frame_id2 = r"Img_0001.tif"
filter_mode = "Img"
# filter_mode = "aharm_thresh"

raw_img_adr = data_adr / video_id / "Img"

fps = 1
winsize = 10  # pixels, interrogation window size in frame A
searchsize = 12  # pixels, search area size in frame B
overlap = 4  # pixels
frame_ids = (0, 1)

STN_thresh = 0.08

piv_param_obj = PIV_params(
    video_path=str(data_adr / video_id / filter_mode),
    segment_mode="other",
    fps=fps,
    window_size=winsize,
    search_size=searchsize,
    overlap_size=overlap,
    stn_threshold=STN_thresh,
    px_per_mm=1500 / 2,
)

In [ ]:
amf_piv_obj = AMF_PIV(piv_param_obj)
amf_piv_obj.plot_raw_images(frame_ids)
amf_piv_obj.plot_segmentation()


In [ ]:
# amf_piv_obj.piv_process(frame_ids, FAKE_OUTLIERS=False)
amf_piv_obj.piv_process_windef(frame_ids)

In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt


def read_and_filter_piv(um_per_px, fps, import_path):
    import_db = pd.read_table(import_path)
    import_db = import_db[import_db["mask"] == 0]
    import_db = import_db[import_db["flags"] == 0]
    import_db["abs"] = np.sqrt(import_db["u"] ** 2 + import_db["v"] ** 2)
    import_db["um/s"] = import_db["abs"] * um_per_px * fps
    import_db["dir"] = np.arctan2(import_db["v"], import_db["u"])
    import_db["speed_x"], import_db["speed_y"] = (
        import_db["u"] * um_per_px * fps,
        import_db["v"] * um_per_px * fps,
    )
    # import_db = import_db[abs(import_db["abs"]) - 0.8 > 0]
    return import_db


def polar2cart(theta, r):
    x = r * np.cos(theta)
    y = r * np.sin(theta)
    return x, y


um_per_px = 0.138
fps = 20
time_point = "0027"

In [ ]:
def save_piv_quiver(time_point):
    import_path = Path(
        rf"U:\test_data\20250122_Plate017\043\OpenPIV_results_8_PIV_output\field_A{time_point}.txt"
    )
    import_db = read_and_filter_piv(um_per_px, fps, import_path)
    fig, ax = plt.subplots(figsize=(5, 7), dpi=200)
    ax.quiver(
        import_db["# x"], import_db["y"], import_db["u"], import_db["v"], scale=300
    )
    ax.set_aspect("equal")
    fig.savefig(rf"U:\test_data\20250122_Plate017\043\quiver_movie\{time_point}")
    fig.clear()


for i in range(2000):
    time_str = f"{i:04}"
    save_piv_quiver(time_str)

In [ ]:
fig, ax = plt.subplot_mosaic(
    [["speed_hist", "density"], ["angle_hist", "density"]], figsize=(8, 4)
)
ax["speed_hist"].hist(import_db["um/s"], bins=50, range=(1, 10))
ax["angle_hist"].hist(import_db["dir"], bins=50, range=(-np.pi, np.pi))
ax["density"].scatter(import_db["speed_x"], import_db["speed_y"], alpha=0.03)
ax["density"].grid(True)
ax["density"].axhline(0, c="black")
ax["density"].axvline(0, c="black")
import_db